In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from varclushi import VarClusHi

In [3]:
df = pd.read_excel('Data- acquisition scorecard.xlsx')

In [4]:
df.rename(columns={'ENQ_1':'days_last_enq','ENQ_2':'days_hl_last_enq','ENQ_3':'#_enq_3mnths','ENQ_4':'#_hle_enq_3mnths','ENQ_5':'#_enq_12mnths','ENQ_6':'#_hle_enq_12mnths','ENQ_7':'Tot_enq','ENQ_8':'Tot_hle_enq','ACCOUNT_9':'#_defaults_3mnths','ACCOUNT_10':'#_defaults_12mnths','ACCOUNT_11':'days_last_acc_open','ACCOUNT_12':'Tot_accounts','ACCOUNT_13':'Tot_hle_accounts','ACCOUNT_14':'Tot_unsecured_loans','ACCOUNT_15':'Tot_live_loans','ACCOUNT_16':'Tot_live_homeloans','ACCOUNT_17':'Tot_live_unsecuredloans','ACCOUNT_18':'Tot_amt_outstanding','ACCOUNT_19':'Tot_secured_outstanding_amt','ACCOUNT_20':'Tot_unsecured_outstanding_amt','ACCOUNT_21':'Avg_days_in_debt'},inplace=True)

In [5]:
df_copy = df.copy()

## Data Preparation

In [6]:
len(df)

3980

In [7]:
df.ID.nunique()

3894

In [8]:
df.drop(df[df.duplicated()].index,inplace=True)

### Missing

In [9]:
df.isnull().sum().sort_values(ascending=False).head(7)

Bank_balance        626
Type_of_industry      5
SEX                   4
Marital_Status        4
Months_in_city        4
Age                   1
ID                    0
dtype: int64

In [10]:
sal_bb = df.loc[df['Customer_type'] == 'SALARIED','Bank_balance'].mean()
se_bb = df.loc[df['Customer_type'] == 'SELF EMPLOYED','Bank_balance'].mean()
ne_bb = df.loc[df['Customer_type'] == 'NON EARNING','Bank_balance'].mean()

In [11]:
df['Bank_balance'] = np.where((df['Customer_type'] == 'SALARIED') & (df['Bank_balance'].isna()),sal_bb,np.where(((df['Customer_type'] == 'SELF EMPLOYED')&(df['Bank_balance'].isna())),se_bb,np.where(((df['Customer_type'] == 'NON EARNING') & (df['Bank_balance'].isna())),ne_bb,df['Bank_balance'])))

In [12]:
df.isnull().sum().sort_values(ascending=False).head(7)

Type_of_industry       5
SEX                    4
Marital_Status         4
Months_in_city         4
Age                    1
ID                     0
Tot_unsecured_loans    0
dtype: int64

In [13]:
df.dropna(inplace=True)

## Variable creation

In [14]:
df.head(1)

,ID,Age,Customer_type,Gross_income,Net_income,SEX,Type_of_industry,Marital_Status,Months_in_city,Months_in_current_job,Org_Type,Bank_balance,Debt_ratio,Target,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt
0,1794,40.000,SALARIED,70891.000,62491.000,MALE,OTHERS,MARRIED,229.000,108,INDIVIDUAL,37806.000,0.190,0,324,0,0,0,1,0,14,0,0,0,1486,9,0,8,2,0,2,275606,0,275606,5311


In [15]:
dt = df.dtypes.to_frame()

In [16]:
non_obj = dt.loc[dt[0] != 'object',:].index.tolist()

In [17]:
non_obj.remove('ID')
non_obj.remove('Target')

In [19]:
for i in non_obj:
    for ii in non_obj:
        if ii != i:
            df.loc[:,'{}/{}'.format(i,ii)] = df[i]/df[ii]
        else:
            continue

In [21]:
dt = df.dtypes.to_frame()

In [22]:
df.head(2)

,ID,Age,Customer_type,Gross_income,Net_income,SEX,Type_of_industry,Marital_Status,Months_in_city,Months_in_current_job,Org_Type,Bank_balance,Debt_ratio,Target,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt,Age/Gross_income,Age/Net_income,Age/Months_in_city,Age/Months_in_current_job,Age/Bank_balance,Age/Debt_ratio,Age/days_last_enq,Age/days_hl_last_enq,Age/#_enq_3mnths,Age/#_hle_enq_3mnths,Age/#_enq_12mnths,Age/#_hle_enq_12mnths,Age/Tot_enq,Age/Tot_hle_enq,Age/#_defaults_3mnths,Age/#_defaults_12mnths,Age/days_last_acc_open,Age/Tot_accounts,Age/Tot_hle_accounts,Age/Tot_unsecured_loans,Age/Tot_live_loans,Age/Tot_live_homeloans,Age/Tot_live_unsecuredloans,Age/Tot_amt_outstanding,Age/Tot_secured_outstanding_amt,Age/Tot_unsecured_outstanding_amt,Age/Avg_days_in_debt,Gross_income/Age,Gross_income/Net_income,Gross_income/Months_in_city,Gross_income/Months_in_current_job,Gross_income/Bank_balance,Gross_income/Debt_ratio,Gross_income/days_last_enq,Gross_income/days_hl_last_enq,Gross_income/#_enq_3mnths,Gross_income/#_hle_enq_3mnths,Gross_income/#_enq_12mnths,Gross_income/#_hle_enq_12mnths,Gross_income/Tot_enq,Gross_income/Tot_hle_enq,Gross_income/#_defaults_3mnths,Gross_income/#_defaults_12mnths,Gross_income/days_last_acc_open,Gross_income/Tot_accounts,Gross_income/Tot_hle_accounts,Gross_income/Tot_unsecured_loans,Gross_income/Tot_live_loans,Gross_income/Tot_live_homeloans,Gross_income/Tot_live_unsecuredloans,Gross_income/Tot_amt_outstanding,Gross_income/Tot_secured_outstanding_amt,Gross_income/Tot_unsecured_outstanding_amt,Gross_income/Avg_days_in_debt,Net_income/Age,Net_income/Gross_income,Net_income/Months_in_city,Net_income/Months_in_current_job,Net_income/Bank_balance,Net_income/Debt_ratio,Net_income/days_last_enq,Net_income/days_hl_last_enq,Net_income/#_enq_3mnths,Net_income/#_hle_enq_3mnths,Net_income/#_enq_12mnths,Net_income/#_hle_enq_12mnths,Net_income/Tot_enq,Net_income/Tot_hle_enq,Net_income/#_defaults_3mnths,Net_income/#_defaults_12mnths,Net_income/days_last_acc_open,Net_income/Tot_accounts,Net_income/Tot_hle_accounts,Net_income/Tot_unsecured_loans,Net_income/Tot_live_loans,Net_income/Tot_live_homeloans,Net_income/Tot_live_unsecuredloans,Net_income/Tot_amt_outstanding,Net_income/Tot_secured_outstanding_amt,Net_income/Tot_unsecured_outstanding_amt,Net_income/Avg_days_in_debt,Months_in_city/Age,Months_in_city/Gross_income,Months_in_city/Net_income,Months_in_city/Months_in_current_job,Months_in_city/Bank_balance,Months_in_city/Debt_ratio,Months_in_city/days_last_enq,Months_in_city/days_hl_last_enq,Months_in_city/#_enq_3mnths,Months_in_city/#_hle_enq_3mnths,Months_in_city/#_enq_12mnths,Months_in_city/#_hle_enq_12mnths,Months_in_city/Tot_enq,Months_in_city/Tot_hle_enq,Months_in_city/#_defaults_3mnths,Months_in_city/#_defaults_12mnths,Months_in_city/days_last_acc_open,Months_in_city/Tot_accounts,Months_in_city/Tot_hle_accounts,Months_in_city/Tot_unsecured_loans,Months_in_city/Tot_live_loans,Months_in_city/Tot_live_homeloans,Months_in_city/Tot_live_unsecuredloans,Months_in_city/Tot_amt_outstanding,Months_in_city/Tot_secured_outstanding_amt,Months_in_city/Tot_unsecured_outstanding_amt,Months_in_city/Avg_days_in_debt,Months_in_current_job/Age,Months_in_current_job/Gross_income,Months_in_current_job/Net_income,Months_in_current_job/Months_in_city,Months_in_current_job/Bank_balance,Months_in_current_job/Debt_ratio,Months_in_current_job/days_last_enq,Months_in_current_job/days_hl_last_enq,Months_in_current_job/#_enq_3mnths,Months_in_current_job/#_hle_enq_3mnths,Months_in_current_job/#_enq_12mnths,Months_in_current_job/#_hle_enq_12mnths,Months_in_current_job/Tot_enq,Months_in_current_job/Tot_hle_enq,Months_in_current_j

In [23]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [24]:
non_obj = dt.loc[dt[0] != 'object',:].index.tolist()

In [25]:
non_obj.remove('ID')
non_obj.remove('Target')

In [26]:
non_obj_iv = pd.DataFrame()
for i in non_obj:
    temp = df.loc[:,[i,'Target']]
    temp['qcut'] = pd.qcut(temp[i],10,duplicates='drop')
    res = pd.crosstab(temp['qcut'],temp['Target'],normalize='columns').assign(woe=lambda x:np.log(x[0]/x[1])).assign(iv=lambda x:np.sum(x['woe']*(x[0]-x[1])))
    res['name'] = i
    non_obj_iv = pd.concat([non_obj_iv,res])

In [27]:
obj = dt.loc[dt[0] == 'object'][0].index.tolist()

In [28]:
obj_iv = pd.DataFrame()
for i in obj:
    res = pd.crosstab(df[i],df['Target'],normalize='columns').assign(woe=lambda x:np.log(x[0]/x[1])).assign(iv=lambda x:np.sum(x['woe']*(x[0]-x[1])))
    res['name'] = i
    obj_iv = pd.concat([obj_iv,res])

In [29]:
iv = pd.concat([non_obj_iv,obj_iv])

In [30]:
iv.rename(columns={0:'%good',1:'%bad'},inplace=True)

In [30]:
iv.drop(iv[iv['iv'].isin([np.inf,-np.inf])].index,inplace=True)

In [31]:
iv.drop_duplicates(subset='name',inplace=True)

In [32]:
iv.sort_values(by='iv',ascending=False,inplace=True)

In [31]:
iv.head(7)

Target,%good,%bad,woe,iv,name
"(20.999, 28.0]",0.118,0.115,0.022,0.057,Age
"(28.0, 31.0]",0.114,0.092,0.216,0.057,Age
"(31.0, 33.0]",0.093,0.092,0.013,0.057,Age
"(33.0, 35.0]",0.080,0.129,-0.474,0.057,Age
"(35.0, 38.0]",0.131,0.095,0.326,0.057,Age
"(38.0, 40.0]",0.075,0.064,0.151,0.057,Age
"(40.0, 43.0]",0.120,0.129,-0.073,0.057,Age


In [32]:
iv.to_csv('IV_complete.csv')

In [34]:
list(set(iv.loc[iv['iv'] == 0,'name'].values.tolist()))

[]

In [35]:
fin_divs = list(set(iv.loc[iv['iv']>0.3,'name'].values))

In [36]:
og_cols = df_copy.columns

In [37]:
df = df[[*og_cols,*fin_divs]]

In [38]:
df.head(1)

,ID,Age,Customer_type,Gross_income,Net_income,SEX,Type_of_industry,Marital_Status,Months_in_city,Months_in_current_job,Org_Type,Bank_balance,Debt_ratio,Target,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt,days_last_acc_open/#_hle_enq_3mnths,#_enq_12mnths/Tot_live_homeloans,#_enq_3mnths/Tot_hle_accounts,Age/Tot_live_homeloans,#_enq_3mnths/#_defaults_3mnths,Tot_enq/#_defaults_3mnths,#_enq_12mnths/Age,Tot_enq/#_hle_enq_3mnths,days_last_enq/Tot_hle_accounts,#_enq_3mnths/Tot_live_homeloans,Tot_unsecured_outstanding_amt/#_defaults_3mnths,#_enq_3mnths/#_hle_enq_3mnths,Tot_enq/Tot_hle_accounts,days_last_acc_open/#_hle_enq_12mnths,#_enq_12mnths/#_defaults_3mnths,#_enq_12mnths,Tot_secured_outstanding_amt/Tot_live_homeloans,Tot_live_unsecuredloans/#_defaults_3mnths,Bank_balance/#_defaults_3mnths
0,1794,40.000,SALARIED,70891.000,62491.000,MALE,OTHERS,MARRIED,229.000,108,INDIVIDUAL,37806.000,0.190,0,324,0,0,0,1,0,14,0,0,0,1486,9,0,8,2,0,2,275606,0,275606,5311,NaN,NaN,NaN,NaN,NaN,NaN,0.025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


In [39]:
df.shape

(3885, 54)

In [40]:
fin_nulls = pd.DataFrame(df.isnull().sum().sort_values(ascending=False))

In [41]:
list(fin_nulls.loc[fin_nulls[0]==3335].index)

['Bank_balance/#_defaults_3mnths',
 'Tot_unsecured_outstanding_amt/#_defaults_3mnths',
 'Tot_live_unsecuredloans/#_defaults_3mnths',
 '#_enq_12mnths/#_defaults_3mnths',
 '#_enq_3mnths/#_defaults_3mnths',
 'Tot_enq/#_defaults_3mnths']

In [42]:
df.drop(columns=list(fin_nulls.loc[fin_nulls[0]==3335].index),inplace=True)

In [43]:
df.shape

(3885, 48)

In [44]:
df.fillna(-1,inplace=True)

In [45]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [46]:
df = pd.get_dummies(df)

In [47]:
cols = df.columns.to_list()
cols.remove('ID')
cols.remove('Target')

In [48]:
model_performance = pd.DataFrame()
for i in cols:
    for ii in cols:
        if i != ii:
            x = df.loc[:,[i,ii]]
            y = df.loc[:,'Target']
            x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)
            model = DecisionTreeClassifier(criterion='entropy')
            model.fit(x_train,y_train)
            pred = model.predict(x_test)
            c = {}
            c['var1'] = [i]
            c['var2'] = [ii]
            c['accuracy'] = [accuracy_score(y_test,pred)]
            model_performance = pd.concat([model_performance,pd.DataFrame.from_dict(c)])

In [49]:
model_performance.to_csv('DecisionTree_check.csv')

In [50]:
iv.to_csv('IV.csv')

## Random Forest

In [51]:
rf_model = RandomForestClassifier(criterion='entropy')

In [52]:
x = df.drop(columns=['ID','Target'])
y = df.Target

In [53]:
rf_model.fit(x,y)

RandomForestClassifier(criterion='entropy')

In [54]:
rf_df = pd.DataFrame(rf_model.feature_importances_,index=rf_model.feature_names_in_)

In [55]:
rf_df.to_csv('RandomForest_FeatureImportance.csv')

## Variable Clustering

In [57]:
var_clus_df = df.drop(columns='Target')

In [62]:
var_clus_df.head()

,ID,Age,Gross_income,Net_income,Months_in_city,Months_in_current_job,Bank_balance,Debt_ratio,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt,days_last_acc_open/#_hle_enq_3mnths,#_enq_12mnths/Tot_live_homeloans,#_enq_3mnths/Tot_hle_accounts,Age/Tot_live_homeloans,#_enq_12mnths/Age,Tot_enq/#_hle_enq_3mnths,days_last_enq/Tot_hle_accounts,#_enq_3mnths/Tot_live_homeloans,#_enq_3mnths/#_hle_enq_3mnths,Tot_enq/Tot_hle_accounts,days_last_acc_open/#_hle_enq_12mnths,#_enq_12mnths,Tot_secured_outstanding_amt/Tot_live_homeloans,Customer_type_NON EARNING,Customer_type_SALARIED,Customer_type_SELF EMPLOYED,SEX_FEMALE,SEX_MALE,Type_of_industry_AGRICULTURE,Type_of_industry_AUTO COMPONENTS,Type_of_industry_AUTOMOTIVE,Type_of_industry_BANKING AND FINANCE,Type_of_industry_CEMENT AND CEMENT PRODUCTS,Type_of_industry_CERAMIC TILES AND SANITARY WARE,Type_of_industry_CHEMICALS,Type_of_industry_COAL,Type_of_industry_COMMUNICATION,Type_of_industry_CONSTRUCTION,Type_of_industry_CONSTRUCTION IRRIGATION PROJECTS,Type_of_industry_CONSTRUCTION REAL ESTATE AND INDUSTRIAL,Type_of_industry_CONSUMER ELECTRONICS,Type_of_industry_CONSUMER PRODUCTS,Type_of_industry_CONTAINERS AND PACKING,Type_of_industry_EDUCATION,Type_of_industry_ELECTRICAL EQUIPMENT AND COMPONENTS,Type_of_industry_ELECTRICAL MACHINERY,Type_of_industry_ELECTRONICS,Type_of_industry_ENGINEERING,Type_of_industry_ENTERTAINMENT AND LEISURE,Type_of_industry_ERW STEEL PIPES,Type_of_industry_FABRICATED METAL PRODUCT,Type_of_industry_FMCG,Type_of_industry_FOOD PRODUCTS,Type_of_industry_GEMS AND JEWELLRY,Type_of_industry_GIFT ITEMS AND HANDICRAFTS,Type_of_industry_GLASS AND GLASS PRODUCTS,Type_of_industry_GOVERNMENT,Type_of_industry_HEALTHCARE AND HEALTHCARE PROVIDERS,Type_of_industry_HOSPITALS AND NURSING HOMES,Type_of_industry_HOTELS AND RESORTS,Type_of_industry_INDUSTRIAL EQUIPMENT,Type_of_industry_INSTITUTIONS AND TRUST,Type_of_industry_INSURANCE,Type_of_industry_IT AND COMMUNICATION,Type_of_industry_IT HARDWARE,Type_of_industry_IT SERVICES,Type_of_industry_LEATHER,Type_of_industry_LIGHT ENGINEERING,Type_of_industry_MACHINE TOOLS,Type_of_industry_MACHINERY AND EQUIPMENT,Type_of_industry_MEDIA,Type_of_industry_MEDIA ADVERTISING,Type_of_industry_MEDIA MOVIES AND ENTERTAINMENT,Type_of_industry_MEDICAL AND OPTICAL INSTRUMENT,Type_of_industry_MEDICAL EQUIPMENT,Type_of_industry_METALS,Type_of_industry_MILK AND DAIRY PRODUCTS,Type_of_industry_NON METAL MINERAL PRODUCT,Type_of_industry_OIL AND GAS ALLIED,Type_of_industry_OTHER MANUFACTURING,Type_of_industry_OTHER SERVICES,Type_of_industry_OTHERS,Type_of_industry_PAINTS,Type_of_industry_PAPER & PULP,Type_of_industry_PETROLEUM PRODUCTS,Type_of_industry_PHARMACEUTICALS BULK DRUGS,Type_of_industry_PHARMACEUTICALS FORMULATIONS,Type_of_industry_PHOTOGRAPHIC EQUIPMENT AND ALLIED PRODUCTS,Type_of_industry_PLASTIC AND PLASTIC PRODUCTS,Type_of_industry_PLASTIC PIPE AND PIPE FITTINGS,Type_of_industry_PLASTIC PRODUCTS,Type_of_industry_PLYWOOD AND LAMINATES,Type_of_industry_POULTRY,Type_of_industry_PRINT MANUFACTURE RECORD MEDIA,Type_of_industry_PRINTING AND PACKAGING,Type_of_industry_PROFESSIONAL SERVICES,Type_of_industry_SECURITY SERVICES,Type_of_industry_STATIONARY AND OTHER SUPPLIES,Type_of_industry_STEEL PIG IRON SPONGE IRON,Type_of_industry_TEA,Type_of_industry_TEXTILE READY MADE GARMENTS,Type_of_industry_TEXTILE WEAVING KNITTING AND PROCESSING,Type_of_industry_TEXTILES,Type_of_industry_TRANSPORT LOGISTICS,Type_of_industry_TRANSPORT OPERATOR,Type_of_industry_TRAVEL AGENTS AND TOUR OPERATORS,Type_of_industry_UTILITY SERVICES,Type_of_industry_WOOD AND WOOD PRODUCTS,Marital_Status_DIVORCED,Marital_Status_MARRIED,Marital_Status_SEPARATED,Marital_Status_

In [59]:
var_clus = VarClusHi(var_clus_df)

In [60]:
var_clus.varclus()

ValueError: shapes (145,) and (4,145) not aligned: 145 (dim 0) != 4 (dim 0)

In [63]:
df.shape

(3885, 144)